In [1]:
import sys
import os
# Append the library path to PYTHONPATH, so library can be imported.
sys.path.append(os.path.dirname(os.getcwd()))
import shutil
import numpy as np
import pandas as pd

from library import stoxx as st
from library import common as cm

In [2]:
%run setup.py
%load_ext autoreload
%autoreload 2

np.random.seed(RANDOM_SEED)

In [3]:
clean_dir = DATA_DIR + 'CleanData/'

In [4]:
df = pd.read_csv(clean_dir + f'Options_with_Liquidity_{FREQ}.csv', index_col=0,
                parse_dates=['ExecuteTime0', 'FuturesExpiry', 'Expiry', 'date', 'DateTime_LIQ'],
                dtype={'SecurityType': 'category',
                        'TrdType': 'category',
                        'AggressorSide': 'category',
                          'cp_int': 'int'})

In [5]:
df.shape

(641234, 57)

The `SIDE` in the liquidty file is the same as the `AggressorSide`, due to the way the data is merged.

In [6]:
df = df.loc[df['LIQUIDITY_IND'].notna()]

In [7]:
df.shape

(376791, 57)

In [8]:
bl = df['ExecuteTime0'] - df['DateTime_LIQ'] < LAG_LIQ_TOL
df = df.loc[bl]

df.shape

(226504, 57)

In [9]:
max_period = max([int(s[6:]) for s in df.columns if 'period' in s])

There are 4 extra liquidty information. `LIQUIDITY_IND` is the most hopeful one.

In [10]:
regr_dir = res_dir + f'Regression_LIQ_{LAG_LIQ_TOL_f}/'

In [11]:
regr_dir

'C:\\Users\\Weiguan\\Dropbox\\Research\\DeepHedging\\Data\\Euroxx\\Result/FREQ=1H_OTM=True_MINM=0.8_MAXM=1.5_Permute=False_VIX=False_WINDOW=90D_AGGSIDE=False_MATCHING=0.1H_CLOSENESS=False/Regression_LIQ_0.25/'

In [12]:
if os.path.exists(regr_dir):
    shutil.rmtree(regr_dir)
os.makedirs(regr_dir)

print(f'Results are saved at:\n{regr_dir}')

Results are saved at:
C:\Users\Weiguan\Dropbox\Research\DeepHedging\Data\Euroxx\Result/FREQ=1H_OTM=True_MINM=0.8_MAXM=1.5_Permute=False_VIX=False_WINDOW=90D_AGGSIDE=False_MATCHING=0.1H_CLOSENESS=False/Regression_LIQ_0.25/


In [13]:
def save_tables(res, dirs):
    if res is not None:
        res['df_coef'].to_csv(f'{dirs}coef.csv')
        res['df_fit_std'].to_csv(f'{dirs}std.csv')
    
    ins = cm.LocalInspector({'MSHE': f'{dirs}'})
    ins.load_real_pnl()
    ins.compare_period().to_csv(f'{dirs}period_pnl.csv')

In [14]:
if not PERMUTE:
    sub_regr_dir = f'{regr_dir}Vega_LIQ/'
    kwargs = {
        'vix': VIX,
        'features': ['vega_n', 'LIQUIDITY_IND'],
        'max_period': max_period,
        'sub_res': f'{sub_regr_dir}pnl/',
        'pnl_path': f'{sub_regr_dir}pnl/pnl.csv',
        'df': df,
        'hull_white': True,
        'agg_side': AGG_SIDE_FLAG
    }

    res = cm.run_store_lin(**kwargs)
    save_tables(res, sub_regr_dir)

In [15]:
f'{sub_regr_dir}pnl/pnl.csv'

'C:\\Users\\Weiguan\\Dropbox\\Research\\DeepHedging\\Data\\Euroxx\\Result/FREQ=1H_OTM=True_MINM=0.8_MAXM=1.5_Permute=False_VIX=False_WINDOW=90D_AGGSIDE=False_MATCHING=0.1H_CLOSENESS=False/Regression_LIQ_0.25/Vega_LIQ/pnl/pnl.csv'

In [16]:
pnl = pd.read_csv(f'{sub_regr_dir}pnl/pnl.csv')

In [17]:
(pnl['PNL']**2).mean()

0.0008389546397758429

In [18]:
if not PERMUTE:
    sub_regr_dir = f'{regr_dir}Vega/'
    kwargs = {
        'vix': VIX,
        'features': ['vega_n'],
        'max_period': max_period,
        'sub_res': f'{sub_regr_dir}pnl/',
        'pnl_path': f'{sub_regr_dir}pnl/pnl.csv',
        'df': df,
        'hull_white': True,
        'agg_side': AGG_SIDE_FLAG
    }

    res = cm.run_store_lin(**kwargs)
    save_tables(res, sub_regr_dir)

In [19]:
f'{sub_regr_dir}pnl/pnl.csv'

'C:\\Users\\Weiguan\\Dropbox\\Research\\DeepHedging\\Data\\Euroxx\\Result/FREQ=1H_OTM=True_MINM=0.8_MAXM=1.5_Permute=False_VIX=False_WINDOW=90D_AGGSIDE=False_MATCHING=0.1H_CLOSENESS=False/Regression_LIQ_0.25/Vega/pnl/pnl.csv'

In [20]:
pnl = pd.read_csv(f'{sub_regr_dir}pnl/pnl.csv')

In [21]:
(pnl['PNL']**2).mean()

0.000837887452967732

In [22]:
if not PERMUTE:
    sub_regr_dir = f'{regr_dir}Delta_Vega_LIQ/'
    kwargs = {
        'vix': VIX,
        'features': ['delta_bs', 'vega_n', 'LIQUIDITY_IND'],
        'max_period': max_period,
        'sub_res': f'{sub_regr_dir}pnl/',
        'pnl_path': f'{sub_regr_dir}pnl/pnl.csv',
        'df': df,
        'hull_white': False,
        'agg_side': AGG_SIDE_FLAG
    }

    res = cm.run_store_lin(**kwargs)
    save_tables(res, sub_regr_dir)

In [23]:
print(f'{sub_regr_dir}pnl/pnl.csv')
pnl = pd.read_csv(f'{sub_regr_dir}pnl/pnl.csv')
(pnl['PNL']**2).mean()

C:\Users\Weiguan\Dropbox\Research\DeepHedging\Data\Euroxx\Result/FREQ=1H_OTM=True_MINM=0.8_MAXM=1.5_Permute=False_VIX=False_WINDOW=90D_AGGSIDE=False_MATCHING=0.1H_CLOSENESS=False/Regression_LIQ_0.25/Delta_Vega_LIQ/pnl/pnl.csv


0.0008379450031569791